In [162]:
import pandas as pd
import yahoo_fantasy_api as yfa
from yahoo_oauth import OAuth2
#import gspread

In [164]:
# # Google Sheets setup
# scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
#          "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name('GoogleApi.json', scope)
# client = gspread.authorize(creds)

# # Export DataFrame to Google Sheets
# sheet_name = "Fantasy League Stats"
# spreadsheet = client.create(sheet_name)
# worksheet = spreadsheet.sheet1
# worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # Upload the DataFrame data


In [186]:
#Auth
sc = OAuth2(None, None, from_file="YahooAuth.json")

[2024-11-11 16:48:31,963 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-11-11 16:48:31,995 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2844.5123856067657
[2024-11-11 16:48:31,996 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [166]:
#Get Game object
gm = yfa.Game(sc, 'nba')

In [167]:
leagues = gm.league_ids()
league = gm.to_league(leagues[-1])

In [173]:
#Dict of categories
categories = {
    "5": "FG%",
    "8": "FT%",
    "10": "3PM",
    "12": "PTS",
    "15": "REB",
    "16": "AST",
    "17": "STL",
    "18": "BLK",
    "19": "TO"
}

In [184]:
all_weeks_data = []

# Loop over each week to gather data
for week in range(1, 24):  # Weeks 1 to 23
    # Fetch matchups data for the specific week
    data = league.matchups(week=week)['fantasy_content']['league'][1]['scoreboard']['0']['matchups']
    
    # Iterate over each matchup and team to extract stats
    for matchup_id, matchup_data in data.items():
        # Check that matchup_data is a dictionary
        if not isinstance(matchup_data, dict):
            continue  # Skip non-dictionary entries

        matchup = matchup_data.get('matchup')
        if not matchup:
            continue

        # Go through each team in the matchup, ensure '0' key exists and is a dictionary
        teams = matchup.get('0', {}).get('teams', {})
        if not isinstance(teams, dict):
            continue

        for team_key, team_data in teams.items():
            # Check that team_data is a dictionary and has the expected structure
            if not isinstance(team_data, dict) or 'team' not in team_data:
                continue
            if not isinstance(team_data['team'], list) or len(team_data['team']) < 2:
                continue

            team = team_data['team'][0]  # Access the team list
            team_name = team[2]['name']  # Extract team name

            # Initialize the row with the team name and the week
            team_row = {'Manager': team_name, 'Week': f"{week}"}

            # Fetch team stats for the week
            stats = team_data['team'][1].get('team_stats', {}).get('stats', [])
            
            # Extract relevant stats by stat_id
            for stat in stats:
                stat_id = stat['stat']['stat_id']
                if stat_id in categories:
                    stat_name = categories[stat_id]
                    team_row[stat_name] = stat['stat']['value']

            # Get the games played and remaining games
            games_info = team_data['team'][1].get('team_remaining_games', {}).get('total', {})
            games_played = games_info.get('completed_games', 0)
            remaining_games = games_info.get('remaining_games', 0)
            
            # Calculate total games and format as "games played / total games"
            total_games = games_played + remaining_games
            team_row['Games Played'] = f"{games_played}/{total_games}"

            # Add team row to the list for all weeks
            all_weeks_data.append(team_row)

# Convert the list of all weeks data to a single DataFrame
all_data = pd.DataFrame(all_weeks_data)

In [185]:
all_data

,Manager,Week,FG%,FT%,3PM,PTS,REB,AST,STL,BLK,TO,Games Played
0,Ducksnation,1,.441,.813,58,473,180,98,32,31,67,37/37
1,The Good Gordons,1,.484,.775,56,687,238,172,33,25,95,37/37
2,GG no re,1,.469,.800,54,624,223,111,41,32,81,35/35
3,Thomas's Team,1,.465,.724,73,690,248,132,33,27,77,37/37
4,Goat Club,1,.493,.789,72,562,214,132,33,22,72,36/36
...,...,...,...,...,...,...,...,...,...,...,...,...
195,Bog's Tots,20,,,,,,,,,,0/36
196,The Good Gordons,20,,,,,,,,,,0/35
197,Fab's Fascinating Team,20,,,,,,,,,,0/37
198,Felix’s team,20,,,,,,,,,,0/33
